# UR5 Simulation forward kinematics
**Author** Eduardo Gomez

Using the homogeneos matrix method, predict the foward kinematics of the UR5 

In [1]:
# Import libraries
import zmqRemoteApi
import numpy as np
from time import sleep
from pathlib import Path
import pandas as pd
import utils
from utils import print_rounded_np
from IPython.display import display, Markdown, Latex

## Setup simulation

To set up the simulation, open the `./sim/ur5-simulation.ttt` file. Then run the following code: 

> If there is an error, either the file is not opened or another script is controlling it

In [2]:
# Create a client to get connected to the zmqremoteApi sernver from CoppeliaSim
client = zmqRemoteApi.RemoteAPIClient()

# Get the remote object 'sim'
sim = client.getObject('sim')


Here, we set up the id for controlling the robot

In [3]:
UR5_sim_obj_id = "/UR5"
base_id = sim.getObject(UR5_sim_obj_id)

# the last joint is the rotor for the end effector
# does not affect the final position
no_joints = 6
joint_sim_names = [None for _ in range(6)]

joint_sim_names[0] = f"{UR5_sim_obj_id}/joint"

for i in range(1,len(joint_sim_names)):
    link_joint_str = "/link/joint" * i
    joint_sim_names[i] = f"{UR5_sim_obj_id}{link_joint_str}"


In [4]:
# Assert that names have id. Error is thrown if id doesnt exists
joint_id = [
    sim.getObject(name) for name in joint_sim_names
]

## Setup Data

This is the general logic on how to set up the data

1. Set up the joints to have 0 deg
2. Pair each set of joints as a "Link"
3. For each Link, get the transformation matrix from the first joint to the next
4. Multiply every transformation matrix in order to get a transformation matrix from the base to the last joint

In [5]:
joints = [utils.Joint(sim,_id) for _id in joint_id]

links = [utils.Link(sim, prev, _next) for prev, _next in utils.window_iter(joints,2)]

robot_arm = utils.RobotArm(links)


In [11]:
print(joints[0].position)
print(joints[-1].position)

print(links[0].initial_homogeneous_matrix)
print(links[1].initial_homogeneous_matrix)

[ 0. -0.  0.]
[-0.0956  0.      0.9781]
[[-0.      -0.      -1.      -0.0703 ]
 [ 0.       1.      -0.       0.     ]
 [ 1.       0.      -0.       0.06605]
 [ 0.       0.       0.       1.     ]]
[[1.     0.     0.     0.4251]
 [0.     1.     0.     0.    ]
 [0.     0.     1.     0.    ]
 [0.     0.     0.     1.    ]]


## Resting angle prediction 

In [6]:

predicted = robot_arm.get_predicted_position([0,0,0,0,0])

print("Predicted")
print_rounded_np(predicted[0:3])

print("Actual")
print_rounded_np(joints[-1].position)


Predicted
[-0.0956  0.      0.978 ]
Actual
[-0.0956  0.      0.9781]


## Variable angle prediction

In [7]:

AnglePositions_T = tuple[int,int,int,int,int]

angles_to_try : list[AnglePositions_T] = [
    (20,30,15,5,45),
    (0,-90, 90,0,90),
    (0,10,20, 30,40),
]


errors = []
rel_err = []

table_txt = """
|Run|Angle Set|Predicted position (x,y,z)|Actual position (x,y,z)
|---|--------|-----------------|--------------|
"""
with utils.start_simulation(sim):
    robot_arm.reset_arm()
    sleep(1)
    
    for idx, sets in enumerate(angles_to_try):
        predicted = robot_arm.get_predicted_position(sets)[0:3]

        robot_arm.set_position_arm(sets)
        sleep(1)
        
        actual = joints[-1].position
        
        abs_error = predicted - actual
        errors.append(abs_error)
        rel_err.append(np.divide(abs_error,actual))
        
        table_txt += f"|{idx + 1}|{sets}|{np.round(predicted, 4)}|{np.round(actual, 4)}|\n"

display(Markdown(table_txt))





|Run|Angle Set|Predicted position (x,y,z)|Actual position (x,y,z)
|---|--------|-----------------|--------------|
|1|(20, 30, 15, 5, 45)|[-0.2884  0.5005  0.7646]|[-0.2886  0.5009  0.7643]|
|2|(0, -90, 90, 0, 90)|[-0.11   -0.4107  0.553 ]|[-0.11   -0.4109  0.5528]|
|3|(0, 10, 20, 30, 40)|[-0.099   0.3566  0.8637]|[-0.099   0.3568  0.8635]|


In [8]:

errors = np.abs(np.array(errors))
rel_err = np.abs(np.array(rel_err))

abs_error_mean = errors.mean()
rel_error_mean = rel_err.mean()

print("Mean of abs error: ", np.round(abs_error_mean,4), "mts")
print("Mean of rel error: ", np.round(rel_error_mean,4))
print("Mean of % error: ", np.round(rel_error_mean * 100,4), "%")

Mean of abs error:  0.0002 mts
Mean of rel error:  0.0004
Mean of % error:  0.0434 %


# Conclusions

My conclusion: 

DH parameters would represent an easier way to store all the foward kinematics of the robot. However, due to the UR and Coppelia sim not beign in sync, it was really problematic. 

However, using the most direct approach for the Homogeneous matrix, provides a very large simplification on predicting new positions. 

Maybe the abstraction layer I build was not necesary, but it helped me understand better the flow of th app. 

## What would you like to learn/add to this assignment if you had to complete it just for fun and for the sole intention of learning more about this topic/simulator?

I would love to learn how to use the simulator and how to use different arms. Also, I would like to create a python GUI to interact in real time with the simulation. 

# Reference

- The material seen in class for the Homogeneous matrixes